In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import time
from pprint import pprint
import pandas as pd
from selenium.webdriver.chrome.options import Options
import requests
import json
from config import geoapify_key
import re
import numpy as np


browser = Browser('chrome')
browser.driver.maximize_window()

ModuleNotFoundError: No module named 'config'

In [2]:

# Load the web page
url = "https://www.lcbo.com/en/products#t=clp-products&sort=relevancy&layout=card&f:@lcbo_current_offer=[On%20Sale]&f:@ec_category=[Wine]"
browser.visit(url)

# To select "English" as preffered language
time.sleep(3)
results = browser.links.find_by_text('English')
results.click()
time.sleep(3)

html = browser.html
soup=BeautifulSoup(html, 'html.parser')

NameError: name 'browser' is not defined

In [3]:
# Removes the cookie popup
cookie = browser.find_by_id('btn-cookie-allow').click()

In [4]:
iteration = 0
while True:
    try:     
        # Wait for 1 second
        time.sleep(2)
        # Click on Load More button
        browser.find_by_id('loadMore').click()
    except:
        break
    print(iteration)
    iteration += 1

0
1
2
3


In [5]:
html = browser.html
    
soup = BeautifulSoup(html, 'html.parser')

all_links = soup.find_all("div", class_ = 'coveo-product-items')

links = []

for link in all_links:
    results = link.a['href']
    links.append(results)        

In [6]:
print(len(links))


203


In [7]:
# div_element = soup.find('div', class_='fotorama__stage__frame')
# if div_element and 'href' in div_element.attrs:
#     link = div_element['href']
#     print(link)
# else:
#     print("Link not found.")

In [8]:
regular_price = []
sale_price = []
name = []
rating = []
onsale = [] #this is the On Sale tag
reviews = []
size = []
description = []
category = []
details_list = []
taste_list = []
images = []


max_iteration2 = 50
iteration2 = 0


for record in links:
    if iteration2 >= max_iteration2:
        break
    url = record
    browser.visit(url)
    time.sleep(2)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
#     time.sleep(1)
#     browser.find_by_css('my_store_close_button').click()

    try:
        div_element = soup.find('div', class_='fotorama__stage__frame')
        if div_element and 'href' in div_element.attrs:
            link = div_element['href']
            images.append(link)
        else:
            print("Link not found.")
    except AttributeError:
        images.append('N/A')
        
    try:
        old_price = soup.find('span', class_ = 'old-price')
        old = old_price.find('span', class_ = 'price')
        regular_price.append(old.find('span', class_ = 'price').text)
    except AttributeError:
        regular_price.append('N/A')

    try:
        on_sale_price = soup.find('span', class_ = 'special-price')
        new_price = on_sale_price.find('span', class_ = 'price')
        sale_price.append(new_price.find('span', class_ = 'price').text)
    except AttributeError:
        sale_price.append('N/A')
    
    try:    
        name.append(soup.find('div', class_ = 'page-title-wrapper').text)
    except AttributeError:
        name.append('N/A')
        
    try:
        onsale.append(soup.find('div', class_ = 'amlabel-text').text)
    except AttributeError:
        onsale.append('N/A')
    
    try:
        rating.append(soup.find('div', class_ = 'bv_avgRating_component_container notranslate').text)
    except AttributeError:
        rating.append('N/A')
    
    try:
        reviews.append(soup.find('div', class_ = 'bv_numReviews_text').text)
    except AttributeError:
        reviews.append('N/A')
    
    try:
        size.append(soup.find('div', class_ = 'lcbo-product-size').text)
    except AttributeError:
        size.append('N/A')
        
    try:
        description.append(soup.find('div', class_ = 'testing_note').text)
    except AttributeError:
        description.append('N/A')
        
    categories_all = soup.find('div', class_ = 'breadcrumbs')
    categories = categories_all.find_all('li', class_ = 'item')
    category_list = []

    for category_record in categories:
        category_list.append(category_record.text)
    category.append(category_list[-2])

    moredetail = soup.find('div', class_ = 'moredetail')
    all_details = moredetail.find_all('li')
    details = {}

    for detail_record in all_details:
        details_value = detail_record.find('div', class_ = 'value').text
        details_name = detail_record.find('div', class_ = 'label').text
        details[details_name] = details_value

    details_list.append(details)
    
#     try:
#         flavour_all = soup.find('div', class_ = 'foodParings pip-info')
#         flavour = flavour_all.find('div', class_ = 'label').text
#         flavour_value = flavour_all.find('div', class_ = 'value').text
#         flavours = {
#             flavour: flavour_value
#         }
#     except AttributeError:
#         flavours = {
#             'Flavour': 'N/A'
#         }
        
    
#     try:
#         all_taste = soup.find('div', class_ = 'foodParings pip-info')
#         li_all = all_taste.find_all('li')

#         for li in li_all:
#             label = li.find('div', class_='label').text.strip()
#             value_element = li.find('div', class_='value')
#             if 'aria-label' in value_element.attrs:
#                 aria_label = value_element['aria-label']
#                 number = re.findall(r'\d+', aria_label)
#                 if number:
#                     taste_list.append({
#                         label: int(number[0])
#                     })
#     except AttributeError:
#         taste_list.append('N/A')
        
    print(iteration2)
    iteration2 += 1



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [9]:
data = {
    'Name': name,
    'Regular Price': regular_price,
    'Sale Price': sale_price,
    'Rating': rating,
    'Reviews': reviews,
    'Size': size,
    'Description': description,
    'Category': category,
    'Details': details_list,
    'Image': images
#     'Flavours': flavours
    
}

# Add the details dictionary to the data dictionary
data['Details'] = details_list
# data['Taste'] = taste_list

# Create a dataframe from the data dictionary
df = pd.DataFrame(data)

details_df = pd.json_normalize(df['Details'])
# taste_df = pd.json_normalize(df['Taste'])

df = pd.concat([df, details_df], axis = 1)
# df2 = pd.concat([df1, taste_df], axis = 1)

df.drop('Details', axis = 1, inplace = True)
# df2.drop('Taste', axis = 1, inplace = True)

#Adding Latitude and Longitude columns to the Data Frame
df['Lat'] = ""
df['Lon'] = ""   

In [10]:
print(len(regular_price))
print(len(sale_price))
print(len(name))
print(len(rating))
print(len(onsale))
print(len(reviews))
print(len(size))
print(len(description))
print(len(category))
print(len(details_list))
print(len(images))
# print(len(flavours))
# print(flavours)

50
50
50
50
50
50
50
50
50
50
50


In [11]:
#API call to get Latitude and Longitude
params = {
    "apiKey":geoapify_key,
    "format":"json"
}

# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

df['Lat'] = None
df['Lon'] = None

for index, country in df['Made In'].to_frame().iterrows():
    location = country['Made In']
    params["text"] = location
    
    response = requests.get(base_url, params=params).json()

    latitude = response["results"][0]["lat"]
    longitude = response["results"][0]["lon"]
    df.loc[index, "Lat"] = latitude
    df.loc[index, "Lon"] = longitude

df.head()


,Name,Regular Price,Sale Price,Rating,Reviews,Size,Description,Category,Image,Alcohol/Vol,Made In,Sugar Content,Varietal,By,Release Date,Lat,Lon
0,Paul Mas Cabernet Sauvignon,$14.95,$12.95,N/A,(0),750 ml bottle,Paul Mas vineyards are in the Languedoc region...,Red Wine,https://aem.lcbo.com/content/dam/lcbo/products...,13.5%,"Languedoc, France",5 g/L,Cabernet Sauvignon,NaN,NaN,47.218868,5.965036
1,Blu Giovello Prosecco,$18.00,$15.50,4.4,(9),750 ml bottle,The prosecco in the distinctive blue bottle. I...,White Sparkling Wine,https://aem.lcbo.com/content/dam/lcbo/products...,11%,Italy,14 g/L,Prosecco,Blu Giovello,NaN,42.638426,12.674297
2,Aveleda Vinho Verde,$15.00,$14.00,4.8,(12),750 ml bottle,Aveleda has been a family-owned business since...,White Wine,https://aem.lcbo.com/content/dam/lcbo/products...,9.5%,"Vinho Verde, Portugal",14 g/L,Vinho Verde,Aveleda,NaN,41.0675,-8.237269
3,Jacob's Creek Moscato Rosé,$16.00,$14.00,4.7,(17),750 ml bottle,"Jacob's Creek Moscato is a fresh and fragrant,...",Rose Wine,https://aem.lcbo.com/content/dam/lcbo/products...,7.5%,Australia,70 g/L,Rose Wines,Jacob's Creek,NaN,-24.776109,134.755
4,Sandbanks Summer Rosé VQA,$14.95,$12.95,N/A,(0),750 ml bottle,"Established in Prince Edward County in 2001, S...",Rose Wine,https://aem.lcbo.com/content/dam/lcbo/products...,11.6%,"Ontario, Canada",27 g/L,Rosé,Sandbanks,NaN,50.000678,-86.000977


In [12]:
df.loc[df['Name'] == ' Allegrini Palazzo della Torre 2019', 'Regular Price']

Series([], Name: Regular Price, dtype: object)

In [13]:
df['Regular Price'] = df['Regular Price'].str.replace('$','')
df['Sale Price'] = df['Sale Price'].str.replace('$','')


df['Savings'] = df['Regular Price'].astype(float) - df['Sale Price'].astype(float)

df['Alcohol/Vol'] = df['Alcohol/Vol'].str.rstrip('%')
df['Alcohol/Vol'] = pd.to_numeric(df['Alcohol/Vol']) /100

df.head()

C:\Users\josed\AppData\Local\Temp\ipykernel_24840\2578000165.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Regular Price'] = df['Regular Price'].str.replace('$','')
C:\Users\josed\AppData\Local\Temp\ipykernel_24840\2578000165.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Sale Price'] = df['Sale Price'].str.replace('$','')


,Name,Regular Price,Sale Price,Rating,Reviews,Size,Description,Category,Image,Alcohol/Vol,Made In,Sugar Content,Varietal,By,Release Date,Lat,Lon,Savings
0,Paul Mas Cabernet Sauvignon,14.95,12.95,N/A,(0),750 ml bottle,Paul Mas vineyards are in the Languedoc region...,Red Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.135,"Languedoc, France",5 g/L,Cabernet Sauvignon,NaN,NaN,47.218868,5.965036,2.0
1,Blu Giovello Prosecco,18.00,15.50,4.4,(9),750 ml bottle,The prosecco in the distinctive blue bottle. I...,White Sparkling Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.110,Italy,14 g/L,Prosecco,Blu Giovello,NaN,42.638426,12.674297,2.5
2,Aveleda Vinho Verde,15.00,14.00,4.8,(12),750 ml bottle,Aveleda has been a family-owned business since...,White Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.095,"Vinho Verde, Portugal",14 g/L,Vinho Verde,Aveleda,NaN,41.0675,-8.237269,1.0
3,Jacob's Creek Moscato Rosé,16.00,14.00,4.7,(17),750 ml bottle,"Jacob's Creek Moscato is a fresh and fragrant,...",Rose Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.075,Australia,70 g/L,Rose Wines,Jacob's Creek,NaN,-24.776109,134.755,2.0
4,Sandbanks Summer Rosé VQA,14.95,12.95,N/A,(0),750 ml bottle,"Established in Prince Edward County in 2001, S...",Rose Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.116,"Ontario, Canada",27 g/L,Rosé,Sandbanks,NaN,50.000678,-86.000977,2.0


In [14]:
df.rename(columns={'Sugar Content' : 'Sugar Content g/L'})

,Name,Regular Price,Sale Price,Rating,Reviews,Size,Description,Category,Image,Alcohol/Vol,Made In,Sugar Content g/L,Varietal,By,Release Date,Lat,Lon,Savings
0,Paul Mas Cabernet Sauvignon,14.95,12.95,N/A,(0),750 ml bottle,Paul Mas vineyards are in the Languedoc region...,Red Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.135,"Languedoc, France",5 g/L,Cabernet Sauvignon,NaN,NaN,47.218868,5.965036,2.00
1,Blu Giovello Prosecco,18.00,15.50,4.4,(9),750 ml bottle,The prosecco in the distinctive blue bottle. I...,White Sparkling Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.110,Italy,14 g/L,Prosecco,Blu Giovello,NaN,42.638426,12.674297,2.50
2,Aveleda Vinho Verde,15.00,14.00,4.8,(12),750 ml bottle,Aveleda has been a family-owned business since...,White Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.095,"Vinho Verde, Portugal",14 g/L,Vinho Verde,Aveleda,NaN,41.0675,-8.237269,1.00
3,Jacob's Creek Moscato Rosé,16.00,14.00,4.7,(17),750 ml bottle,"Jacob's Creek Moscato is a fresh and fragrant,...",Rose Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.075,Australia,70 g/L,Rose Wines,Jacob's Creek,NaN,-24.776109,134.755,2.00
4,Sandbanks Summer Rosé VQA,14.95,12.95,N/A,(0),750 ml bottle,"Established in Prince Edward County in 2001, S...",Rose Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.116,"Ontario, Canada",27 g/L,Rosé,Sandbanks,NaN,50.000678,-86.000977,2.00
5,Bask Cabernet Sauvignon,13.45,12.45,3.0,(6),750 ml bottle,Low in sugar and high in flavour! Bask created...,Red Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.125,Canada,3 g/L,Cabernet Sauvignon,NaN,NaN,61.066692,-107.991707,1.00
6,Bread & Butter Cabernet Sauvignon,19.95,17.95,4.2,(48),750 ml bottle,California has a world-class reputation for gr...,Red Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.135,"California, United States",8 g/L,Cabernet Sauvignon,Bread & Butter,NaN,36.701463,-118.755997,2.00
7,The Ned Sauvignon Blanc,17.10,14.10,3.5,(4),750 ml bottle,The Ned is classic New Zealand-style sauvignon...,White Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.130,"Marlborough, New Zealand",5 g/L,Sauvignon Blanc,The Ned,NaN,-36.784622,174.723989,3.00
8,Adobe Reserva Rosé Organic,13.95,11.95,4.8,(5),750 ml bottle,"A vibrant, mouth-watering, organic rosé from C...",Rose Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.120,Chile,6 g/L,Rose Wines,Adobe,NaN,-31.761336,-71.31877,2.00
9,Francis Coppola Diamond Collection Ivory Labe...,24.95,22.95,4.8,(8),750 ml bottle,"Classically styled Cali Cab featuring cedar, p...",Red Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.145,"California, United States",5 g/L,Cabernet Sauvignon,Francis Coppola,"April 1, 2022",36.701463,-118.755997,2.00


In [15]:
nan_df = df.isna()
df = df.fillna('N/A')
df.head()

,Name,Regular Price,Sale Price,Rating,Reviews,Size,Description,Category,Image,Alcohol/Vol,Made In,Sugar Content,Varietal,By,Release Date,Lat,Lon,Savings
0,Paul Mas Cabernet Sauvignon,14.95,12.95,N/A,(0),750 ml bottle,Paul Mas vineyards are in the Languedoc region...,Red Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.135,"Languedoc, France",5 g/L,Cabernet Sauvignon,N/A,N/A,47.218868,5.965036,2.0
1,Blu Giovello Prosecco,18.00,15.50,4.4,(9),750 ml bottle,The prosecco in the distinctive blue bottle. I...,White Sparkling Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.110,Italy,14 g/L,Prosecco,Blu Giovello,N/A,42.638426,12.674297,2.5
2,Aveleda Vinho Verde,15.00,14.00,4.8,(12),750 ml bottle,Aveleda has been a family-owned business since...,White Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.095,"Vinho Verde, Portugal",14 g/L,Vinho Verde,Aveleda,N/A,41.067500,-8.237269,1.0
3,Jacob's Creek Moscato Rosé,16.00,14.00,4.7,(17),750 ml bottle,"Jacob's Creek Moscato is a fresh and fragrant,...",Rose Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.075,Australia,70 g/L,Rose Wines,Jacob's Creek,N/A,-24.776109,134.755000,2.0
4,Sandbanks Summer Rosé VQA,14.95,12.95,N/A,(0),750 ml bottle,"Established in Prince Edward County in 2001, S...",Rose Wine,https://aem.lcbo.com/content/dam/lcbo/products...,0.116,"Ontario, Canada",27 g/L,Rosé,Sandbanks,N/A,50.000678,-86.000977,2.0


In [16]:
column_lists = {column: df[column].tolist() if not np.issubdtype(df[column].dtype, np.number) else df[column].fillna('N/A').tolist() for column in df.columns}

name_list = column_lists['Name']
regular_price_list = column_lists['Regular Price']
sale_price_list = column_lists['Sale Price']
rating_list = column_lists['Rating']
reviews_list = column_lists['Reviews']
size_list = column_lists['Size']
description_list = column_lists['Description']
category_list = column_lists['Category']
alcohol_list = column_lists['Alcohol/Vol']
madein_list = column_lists['Made In']
by_list = column_lists['By']
varietal_list = column_lists['Varietal']
lat_list = column_lists['Lat']
lon_list = column_lists['Lon']
savings_list = column_lists['Savings']
images_list = column_lists['Image']

lcbo_1 = {
    'name': name_list,
    'regular_price': regular_price_list,
    'sale_price': sale_price_list,
    'rating': rating_list,
    'reviews': reviews_list,
    'size': size_list,
    'description': description_list,
    'category': category_list,
    'alcohol_vol': alcohol_list,
    'made_in': madein_list,
    'by': by_list,
    'varietal': varietal_list,
    'lat': lat_list,
    'lon': lon_list,
    'savings': savings_list,
    'image': images_list
}

with open('lcbo_1.json', 'w') as file:
    json.dump(lcbo_1, file)

In [17]:
column_lists = {column: df[column].tolist() if not np.issubdtype(df[column].dtype, np.number) else df[column].fillna('N/A').tolist() for column in df.columns}

name_list = column_lists['Name']
regular_price_list = column_lists['Regular Price']
sale_price_list = column_lists['Sale Price']
rating_list = column_lists['Rating']
reviews_list = column_lists['Reviews']
size_list = column_lists['Size']
description_list = column_lists['Description']
category_list = column_lists['Category']
alcohol_list = column_lists['Alcohol/Vol']
madein_list = column_lists['Made In']
by_list = column_lists['By']
varietal_list = column_lists['Varietal']
lat_list = column_lists['Lat']
lon_list = column_lists['Lon']
savings_list = column_lists['Savings']
images_list = column_lists['Image']

lcbo_2 = {
    'name': name_list,
    'regular_price': regular_price_list,
    'sale_price': sale_price_list,
    'rating': rating_list,
    'reviews': reviews_list,
    'size': size_list,
    'description': description_list,
    'category': category_list,
    'alcohol_vol': alcohol_list,
    'made_in': madein_list,
    'by': by_list,
    'varietal': varietal_list,
    'lat': lat_list,
    'lon': lon_list,
    'savings': savings_list,
    'image': images_list
}

with open('lcbo_2.json', 'w') as file:
    file.write(pd.DataFrame(lcbo_2).to_json(orient='records'))

In [18]:
with open('lcbo_1.json', 'r') as file1:
    lcbo1 = json.load(file1)

# Read the contents of the second JSON file
with open('lcbo_2.json', 'r') as file2:
    lcbo2 = json.load(file2)

# Merge the two JSON data

merged_data = lcbo2
merged_data.extend(lcbo1)

# Write the merged data to a new JSON file
with open('lcbo_merged.json', 'w') as outfile:
    json.dump(merged_data, outfile)

In [19]:
df.to_csv('lcbo_wines.csv', index = False)